# Najboljše videoigre vseh časov

 **Projektna naloga pri predmetu Programiranje 1**

## 0. Uvod <a name="uvod"></a>

Preden sploh lahko začnemo z analizo podatkov, moramo seveda pripravit vse potrebno. V ta namen uvozimo knjižnico pandas in vnesemo naše tabele.

In [32]:
import pandas as pd

pd.options.display.max_rows = 50

%matplotlib inline

igre = pd.read_csv("igre.csv")
zanri = pd.read_csv("zanri.csv")

In [12]:
igre

,id,naslov,platforma,studio,mesec,leto,metascore,glasovi metascore,userscore,glasovi userscore,oznaka,stevilo igralcev,opis
0,160692,The Legend of Zelda: Ocarina of Time,Nintendo 64,Nintendo,Nov,1998,99.0,22.0,9.1,5723.0,E,1.0,"As a young boy, Link is tricked by Ganondorf, ..."
1,160779,Tony Hawk's Pro Skater 2,PlayStation,Activision,Sep,2000,98.0,19.0,7.4,639.0,T,2.0,As most major publishers' development efforts ...
2,108362,Grand Theft Auto IV,PlayStation 3,Rockstar Games,Apr,2008,98.0,64.0,7.6,3797.0,M,1.0,What does the American Dream mean today? For N...
3,160751,SoulCalibur,Dreamcast,Namco,Sep,1999,98.0,24.0,8.5,321.0,T,2.0,"This is a tale of souls and swords, transcendi..."
4,108363,Grand Theft Auto IV,Xbox 360,Rockstar Games,Apr,2008,98.0,86.0,7.9,3360.0,M,1.0,What does the American Dream mean today? For N...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,215521,Atelier Ayesha: The Alchemist of Dusk,PlayStation 3,Koei Tecmo Games,Mar,2013,70.0,17.0,7.7,88.0,T,1.0,Atelier Ayesha: The Alchemist of Dusk follows ...
9996,182135,Payday: The Heist,PlayStation 3,Sony Online Entertainment,Oct,2011,70.0,30.0,6.4,83.0,M,NaN,Players will embark on a spectacular crime spr...
9997,182132,The Sims 3: Pets,PlayStation 3,Electronic Arts,Oct,2011,70.0,16.0,3.9,15.0,T,1.0,Players take creativity by the reins and play ...
9998,110061,Global Agenda,PC,Hi-Rez Studios,Feb,2010,70.0,16.0,7.8,356.0,T,NaN,Global Agenda is a massively multi-player acti...


In [3]:
zanri

,id,zanr
0,160692,Action Adventure
1,160692,Fantasy
2,160779,Sports
3,160779,Alternative
4,160779,Skateboarding
...,...,...
34181,110061,Massively Multiplayer Online
34182,110061,Massively Multiplayer
34183,110061,Sci-Fi
34184,502606,Action Adventure


## 1. Kaj pomeni "najboljša videoigra"?

Da bo naslov te projektne naloge opravičen, bi se spodobilo, da najprej dorečemo, kaj sploh pomeni, da je igra "najboljša". Igre so na spletni strani Metacritic, s katere sem prenesel podatke, razvrščene po metascore. Tu jih bomo ocenjevali malo drugače.

Kot prvo, bomo ločili igre glede na platformo. Različna strojna oprema in različna implementacija pomeni, da se lahko kvaliteta od platforme do platforme precej razlikuje. Zato bomo v določenih pogledih obravnavali najboljše igre na vsaki platformi posebej.

In [4]:
platforme = igre.platforma.unique()
platforme

array(['Nintendo 64', 'PlayStation', 'PlayStation 3', 'Dreamcast',
       'Xbox 360', 'Wii', 'Xbox One', 'Switch', 'PlayStation 2',
       'PlayStation 4', 'GameCube', 'Xbox', 'PC', 'Wii U',
       'Game Boy Advance', '3DS', 'DS', 'PlayStation Vita', 'PSP',
       'Stadia'], dtype=object)

In [5]:
igre_po_platformi = {}

for platforma in platforme:
    igre_po_platformi[platforma] = igre[igre.platforma == platforma]

igre_po_platformi["PC"]

,id,naslov,platforma,studio,mesec,leto,metascore,glasovi metascore,userscore,glasovi userscore,oznaka,stevilo igralcev,opis
22,103028,Half-Life 2,PC,VU Games,Nov,2004,96.0,81.0,9.1,10760.0,M,1.0,"By taking the suspense, challenge and visceral..."
23,325932,Grand Theft Auto V,PC,Rockstar Games,Apr,2015,96.0,57.0,7.8,6639.0,M,32.0,Los Santos: a sprawling sun-soaked metropolis ...
30,109675,The Orange Box,PC,EA Games,Oct,2007,96.0,34.0,9.2,1803.0,M,32.0,Games included in The Orange Box compilation: ...
36,100055,Half-Life,PC,Sierra Entertainment,Nov,1998,96.0,24.0,9.0,4316.0,M,1.0,Half-Life combines great storytelling in the t...
40,105294,BioShock,PC,2K Games,Aug,2007,96.0,44.0,8.6,4804.0,M,1.0,"Going beyond ""run and gun corridors,"" ""monster..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9981,113122,Company of Heroes: Tales of Valor,PC,THQ,Apr,2009,70.0,42.0,7.2,216.0,M,NaN,Company of Heroes: Tales of Valor is the next ...
9982,107733,Romance of the Three Kingdoms XI,PC,Koei,Sep,2008,70.0,11.0,7.9,49.0,E10+,8.0,Set during the end of China's Han Dynasty and ...
9987,496865,I Am Setsuna,PC,Square Enix,Jul,2016,70.0,16.0,7.2,62.0,E10+,1.0,"Inspired by Chrono Trigger, I am Setsuna is an..."
9992,306747,Brigador,PC,Stellar Jockeys,Jun,2016,70.0,12.0,7.8,40.0,NaN,1.0,"Choose your vehicle, weapons, pilot, and defen..."


Sedaj se lahko lotimo vprašanja kako bomo rangirali igre. Konstruirali bomo lastno metriko za rangiranje iger. Zakaj je to potrebno, se morda sprešuje bralec. Razlogov je več:
* Neobelani podatki so razvrščeni po **metascore** torej ocenah Strokovnjakov. Ne upoštevamo torej mnenja povprečnega uporabnika.
* Nikjer ne upoštevamo stevilo ocen, kar lahko pripelje do patoloških primerov. Igra ki ima 5 glasov in oceno 9.0 intuitivno seveda ni enako dobra, kot igra, ki ima 5000 glasov in oceno 8.5.
* Vendar pa bomo upoštevali, da bo, do neke mere, igra z malo glasovi in dobro oceno boljša od igre z veliko glasovi, ki pravijo  da je zanič.   

Vse to bomo dosegli z našo metriko, **prilagojeno oceno**.
Izračun bo sledeč:
Userscore in metascore bomo prilagodili po naslednji formuli:
O_p = O * (S_g/(S_g + S_sg)), kjer je O prvotna ocena, S_g število glasov in S_sg osrednja vrednost ali mediana števila glasov.

S tem smo dosegli sledeče:
* Igre z visoko oceno in velikim številom glasov, bodo obdržale prestižno oceno
* Igre z nizko oceno in velikim številom glasov, bodo prevtako slabo ocenjene
* Igre z sicer malo glasovi, ampak res dobro oceno, pa bodo lahko povorečno, ali celo nadpovprečno ocenjene. 
* Za igre z nizko oceno in malo glasovi smatramo, da niso dovolj popularne, zato se njihova ocena zmanjša.

Laično povedano; Če je glasov veliko, ocena ostane skoraj enaka. Če pa je glasov malo, se ucena temu primerno prilagodi, pri čimer pa lahko kaka res dobra igra pride v prvo polovico. Opazimo da je mediana nekakšna prelomnica, ki ločuje, kaj pomeni imeti veliko in malo glasov. Kot srednja vrednost bo vedno nekako predstavljala tako mejo. V resnici na veliko ocenjene igre ne vpliva, skušamo urediti igre z malo ocenami. Če bi recimo vzeli namesto mediane aritmetično sredino, bi lahko prišlo do tega, da če je v vzorcu kaka igra z res velikim številom glasov, s tem večini dobrih iger z malo glasovim prepreči dobro oceno, ker je prag prevelik.

Nazadnje zmnožimo vrednosti prilagojenih ocen userscore in metascore, da dobimo končno oceno.
Želimo namreč, da se za dobro igro tako uporabniki kot strokovnjaki strinjajo, da je dobra. Če ena stran meni, da je izdelek zanič, bo precej zmanjšala oceno.

In [56]:
def prilagodi_glasove_userscore(tabela_iger):
    sred_st_glasov = (tabela_iger["glasovi userscore"]).median()
    def prilagodi(ocena, st_glasov):
        return round(((st_glasov) / (st_glasov + sred_st_glasov)) * ocena, 1)
    tabela_iger["prilagojen userscore"] = prilagodi(tabela_iger["userscore"], tabela_iger["glasovi userscore"])

In [57]:
test = igre_po_platformi["PC"].copy()
prilagodi_glasove_userscore(test)

test[test["glasovi userscore"] < 500].sort_values("userscore", ascending=False)

86.0


,id,naslov,platforma,studio,mesec,leto,metascore,glasovi metascore,userscore,glasovi userscore,oznaka,stevilo igralcev,opis,prilagojen userscore
755,102947,Age of Empires II: The Conquerors Expansion,PC,Microsoft Game Studios,Aug,2000,88.0,21.0,9.1,428.0,T,8.0,"The official add-on to the best-selling, real-...",7.6
650,100016,The Curse of Monkey Island,PC,LucasArts,Oct,1997,89.0,9.0,9.0,323.0,E,1.0,"Quick - what has dozens of monkeys, ghost pira...",7.1
177,535705,Hades,PC,Supergiant Games,Sep,2020,92.0,49.0,9.0,309.0,NaN,1.0,Defy the god of death as you hack and slash yo...,7.0
230,100138,Thief: The Dark Project,PC,Eidos Interactive,Nov,1998,92.0,18.0,9.0,397.0,M,1.0,Sneak through the shadows of 12 treacherous mi...,7.4
5524,101265,Desperados: Wanted Dead or Alive,PC,Infogrames,Jul,2001,78.0,12.0,8.9,157.0,T,1.0,Lead Cooper and his gang through 25 levels of ...,5.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6999,532651,Defector,PC,Oculus Studios,Jul,2019,75.0,13.0,3.2,6.0,NaN,1.0,Overcome impossible odds and survive certain d...,0.2
2887,110037,Football Manager Live,PC,Sega,Jan,2009,82.0,23.0,3.0,46.0,NaN,NaN,Football Manager Live is a brand new game from...,1.0
3666,510609,FIFA 18,PC,Electronic Arts,Sep,2017,81.0,8.0,2.7,352.0,E,NaN,Score incredible goals in FIFA 18 as new movem...,2.2
2972,560443,The Sims 4: Eco Lifestyle,PC,Electronic Arts,Jun,2020,82.0,10.0,2.6,20.0,T,NaN,Ready for your Sims to impact their world? Mov...,0.5


Na strani Metacritic, s koder sem pobral podatke, so bile razvrščene po **metascore** oceni, tu gre za oceno "strokovnjakov" oziroma "profesionalnih ocenjevalcev iger". Narekovaji v prejšnji povedi so opravičeni, saj te osebe igre, ko izidejo, preizkusijo in nato podajo svojo oceno. Običajni ljudje, ki pa si vsak teden ne privoščijo nove igre, z izdelkom preživijo več časa in zato podajo bolj dolgotrajni utis. Poglejmo koliko se vrednosti razlikujeta v naših podatkih.

In [ ]:
igre[(100 < igre["glasovi userscore"]) & (igre["glasovi userscore"] < 500)].sort_values("prilagojen userscore")

In [ ]:
igre.sort_values("prilagojen userscore", ascending=True).head(20)

In [ ]:
igre[- igre["prilagojen userscore"] + igre["userscore"] > 0.1]

In [ ]:
igre.plot.scatter(x="metascore", y="userscore")

In [ ]:
for platforma in platforme:
    igre[igre.platforma == platforma].plot.scatter(x="metascore", y="userscore")

In [ ]:
igre.sort_values("leto")

In [ ]:
igre.groupby("naslov").size().sort_values()

In [ ]:
igre.groupby("platforma").size().sort_values()

In [ ]:
igre[(igre.platforma == "PC")].sort_values("leto", ascending=False)

In [ ]:
# igre[(len(igre.naslov) > 10)]
def fun(naslov):
    return "Creed" in naslov
    
igre[(igre["naslov"].apply(fun)) & (igre["platforma"] == "PC")].sort_values("leto")

# Table of contents
1. [Introduction](#introduction)
2. [Some paragraph](#paragraph1)
    1. [Sub paragraph](#subparagraph1)
3. [Another paragraph](#paragraph2)

## This is the introduction <a name="introduction"></a>
Some introduction text, formatted in heading 2 style

## Some paragraph <a name="paragraph1"></a>
The first paragraph text

### Sub paragraph <a name="subparagraph1"></a>
This is a sub paragraph, formatted in heading 3 style
## Another paragraph <a name="paragraph2"></a>
The second paragraph text
